In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband
from kerastuner import Objective
import joblib
import keras
from keras.metrics import Precision, Recall
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

# Define la métrica personalizada F1Score
class F1Score(keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision_result = self.precision.result()
        recall_result = self.recall.result()
        return 2 * ((precision_result * recall_result) / (precision_result + recall_result + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()
        
# Registra la métrica personalizada
tf.keras.utils.get_custom_objects()['F1Score'] = F1Score      

# Cargo el archivo CSV en un DataFrame
df = pd.read_csv("datas.csv")

# Separo las características (X) y las etiquetas (y)
X = df.drop(columns=["adopted"])  # Todas las columnas excepto la columna "adopted"
y = df["adopted"]  # Columna "adopted" como etiqueta

# Divido los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Definir el keras tuner
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                                         activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=[F1Score()])  # Agrega las métricas

    return model

# Define the tuners
tuner_names = ['RandomSearch', 'BayesianOptimization']
tuners = [RandomSearch, BayesianOptimization]

# Perform the hyperparameter search with different tuners
best_models = {}
for tuner_name, tuner_class in zip(tuner_names, tuners):
    print(f"Performing {tuner_name} tuning:")
    tuner = tuner_class(
        build_model,
        objective=Objective("f1_score", direction="max"),
        max_trials=5,
        executions_per_trial=1,
        directory=f'my_dir_{tuner_name.lower()}',
        project_name='helloworld_6')

    tuner.search(X_train, y_train,
                 epochs=15,
                 validation_data=(X_val, y_val))

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Build the final model with the best hyperparameters
    best_model = tuner.hypermodel.build(best_hps)

    # Train the final model
    history = best_model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

    # Evaluate the final model
    evaluation = best_model.evaluate(X_test, y_test)
    print("Test Evaluation:", evaluation)
    

    # Get precision and F1 score from evaluation
    f1 = evaluation[1]  # Assuming F1 score is at index 4
        
    # Save the best model with precision and F1 score in the filename
    model_name = f"best_model_{tuner_name.lower()}_f1_{f1:.4f}.keras"
    best_model.save(model_name)

    # Record the best model and evaluation
    best_models[tuner_name] = {'model': best_model, 'evaluation': evaluation}

# Write the results to a text file
with open('tuning_results.txt', 'w') as f:
    f.write("Tuning Results:\n")
    for tuner_name, result in best_models.items():
        f.write(f"\nTuner: {tuner_name}\n")
        f.write(f"Best Evaluation: {result['evaluation']}\n")
        f.write(f"Best Model: {result['model']}\n")
        f.write("\n")

Performing RandomSearch tuning:
Reloading Tuner from my_dir_randomsearch\helloworld_6\tuner0.json
Epoch 1/15
5107/5121 [============================>.] - ETA: 0s - loss: 0.5409 - f1_score: 0.8289

c:\Users\Franco-SIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


5121/5121 [==============================] - 10s 2ms/step - loss: 0.5407 - f1_score: 0.8290 - val_loss: 0.5311 - val_f1_score: 0.8369
Epoch 2/15
5121/5121 [==============================] - 10s 2ms/step - loss: 0.5282 - f1_score: 0.8324 - val_loss: 0.5321 - val_f1_score: 0.8360
Epoch 3/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5259 - f1_score: 0.8328 - val_loss: 0.5309 - val_f1_score: 0.8379
Epoch 4/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5248 - f1_score: 0.8329 - val_loss: 0.5283 - val_f1_score: 0.8383
Epoch 5/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5234 - f1_score: 0.8341 - val_loss: 0.5287 - val_f1_score: 0.8381
Epoch 6/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5231 - f1_score: 0.8338 - val_loss: 0.5280 - val_f1_score: 0.8380
Epoch 7/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5227 - f1_score: 0.8336 - val_loss: 0.5275 - val_f1_score: 0.8385
E

c:\Users\Franco-SIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


641/641 [==============================] - 1s 844us/step - loss: 0.5267 - f1_score: 0.8381
Test Evaluation: [0.526664137840271, 0.8380616903305054]
Performing BayesianOptimization tuning:
Reloading Tuner from my_dir_bayesianoptimization\helloworld_6\tuner0.json
Epoch 1/15
5111/5121 [============================>.] - ETA: 0s - loss: 0.5435 - f1_score: 0.8318

c:\Users\Franco-SIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


5121/5121 [==============================] - 19s 3ms/step - loss: 0.5434 - f1_score: 0.8318 - val_loss: 0.5283 - val_f1_score: 0.8287
Epoch 2/15
5121/5121 [==============================] - 17s 3ms/step - loss: 0.5324 - f1_score: 0.8337 - val_loss: 0.5281 - val_f1_score: 0.8377
Epoch 3/15
5121/5121 [==============================] - 17s 3ms/step - loss: 0.5298 - f1_score: 0.8341 - val_loss: 0.5264 - val_f1_score: 0.8378
Epoch 4/15
5121/5121 [==============================] - 17s 3ms/step - loss: 0.5282 - f1_score: 0.8347 - val_loss: 0.5259 - val_f1_score: 0.8378
Epoch 5/15
5121/5121 [==============================] - 17s 3ms/step - loss: 0.5272 - f1_score: 0.8354 - val_loss: 0.5256 - val_f1_score: 0.8354
Epoch 6/15
5121/5121 [==============================] - 17s 3ms/step - loss: 0.5268 - f1_score: 0.8354 - val_loss: 0.5259 - val_f1_score: 0.8371
Epoch 7/15
5121/5121 [==============================] - 17s 3ms/step - loss: 0.5255 - f1_score: 0.8360 - val_loss: 0.5242 - val_f1_score: 0.8

c:\Users\Franco-SIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


641/641 [==============================] - 1s 1ms/step - loss: 0.5253 - f1_score: 0.8393
Test Evaluation: [0.5253196954727173, 0.83927983045578]
